In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import technical_indicator as ti
import plotting as pl
import walk_forward as wf

from finta import TA
import pandas_ta as pta

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, r2_score
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
from xgboost.sklearn import XGBClassifier
import datetime as dt

In [ ]:
tickers = ['AAPL', 'MSFT', 'AMZN']
df = yf.download(tickers, period='5y', auto_adjust=True, group_by='ticker')
df.head(200)

In [ ]:
# Plot candle chart
pl.plot_candle_chart(df)

In [ ]:
for period in [5, 10, 20, 50, 100, 200]:
    df['SMA_{}'.format(str(period))] = ti.simple_ma(df['Adj Close'], period=period)
    df['EMA_{}'.format(str(period))] = ti.exp_ma(df['Adj Close'], period=period)
    

plt.figure(figsize=(20,10))
plt.plot(df['EMA_50'], label='50-day EMA')
plt.plot(df['EMA_200'], label='200-day EMA')
plt.plot(df['Adj Close'], label='Adj Close', linewidth=1)
plt.legend()
plt.show()

In [ ]:
df_train = df[:int(df.shape[0]*0.7)]
df_test = df[int(df.shape[0]*0.7):]

# y_train = y[:int(X.shape[0]*0.7)]
# y_test = y[int(X.shape[0]*0.7):]



# Feature Engineering

### Features to be considered:
`1`. **Simple Moving Average**



`2`. Exponential Moving Average

`3`. Bollinger Bands

`4`. Daily return

`5`. Log volume

`6`. Rate of change

`7`. Price difference

`8`. Z-score

`9`. Relative Strengh Index (RSI)
Measures the magnitude of recent price changes. Commonly used in technical analysis to evaluate overbought or oversold conditions in the price of a stock.
This indicator was developed by J.Welles Wilder Jr.
According to this indicator, a stock is considered overbought when the RSI is above 70% and oversold when it is below 30%.

`10`. Moving Average Convergence Divergence (MACD)
MACD is a trend-following momentum indicator that demonstrates the relationship between two moving averages - long-term and short-term. MACD is calculated as 

MACD = EMA(26-period) - EMA(12-period)

MACD is often used together with Relative Strength (RSI) to show a more complete picture of a market.

The signal line is a 9-day EMA of the MACD Line. As a moving average of the indicator, it trails the MACD and makes it easier to spot MACD turns.

`11`. Stochastic Oscillator

The Stochastic Oscillator is a momentum indicator that compares a specific closing price of a security to its high-low range over a certain period of time. First developed in the 1950s, it is a popular momentum indicator to identify overbought and oversold conditions of an asset. 
	  
`12`. Accumulation/Distribution Indicator (A/D)

`13`. Average True Range(ATR)
ATR is a market volatility indicator. Introduced by market technician J.Welles Wilder Jr., this indicator measures market volatility by decomposing the complete range of a security price for that period.

`14`. Vortex Indicator
The Vortex Indicator consists of two oscillating lines: one to capture the upward trend movement and the other one to identify the downward price movement.

`15`. Ease of Movement

`16`. Commodity Channel Index (CCI)
A momentum-based oscillator used to identify an asset that is reaching a condition of being oversold or overbought.

`17`. On-Balance Volume (OBV)
A momentum-based indicator using volume flow to predict changes in asset price. First developed by Joseph Granville, he believed that the volume was the major force behind markets. As explained in his 1963 book Granville's New Key to Stock Market Profits, when volume increases strongly without a signficant change in the price of an asset, the price will eventually jump upwards or downwards. Granville theorised that volume precedes price. 

__How to interpret.__

We should expect prices to increase if OBV is rising while prices are either flat or moving down, and vice versa.

# References

https://www.investopedia.com/terms/r/rsi.asp
https://www.investopedia.com/terms/m/macd.asp
https://altfins.com/knowledge-base/macd-line-and-macd-signal-line/
https://www.investopedia.com/terms/s/stochasticoscillator.asp
https://www.investopedia.com/terms/a/accumulationdistribution.asp
https://www.investopedia.com/terms/a/atr.asp
https://www.investopedia.com/articles/active-trading/072115/understand-vortex-indicator-trading-strategies.asp
https://www.investopedia.com/terms/e/easeofmovement.asp
https://www.investopedia.com/terms/c/commoditychannelindex.asp
https://www.investopedia.com/terms/o/onbalancevolume.asp


The 5-, 10-, 20- and 50-day moving averages are frequently utilized to identify near-term trend changes.

In [ ]:
feature_names = ['DAILY_RETURN', 'DIFF', 'VOLUME_LOG', 'f05', 'f06']
col_names = pd.MultiIndex.from_product([tickers, feature_names])
features = pd.DataFrame(index=df.index, columns=col_names).sort_index()

for ticker in tickers:
    features.loc[:, (ticker, 'DAILY_RETURN')] = ti.daily_return(df[ticker]['Close'])
    features.loc[:, (ticker, 'DIFF')] = ti.price_diff(df[ticker]['Close'])
    features.loc[:, (ticker, 'VOLUME_LOG')] = df[ticker]['Volume'].apply(np.log)
    features.loc[:, (ticker, 'DIFF')] = ti.price_diff(df[ticker]['Close'])
    
    # 5-day moving average of volume
    features.loc[:, (ticker, 'f05')] = ti.simple_ma(features[ticker]['VOLUME_LOG'], period=5)
    
    # Close vs 50-day EMA 
    features.loc[:, (ticker, 'f06')] = df[ticker]['Close'] / ti.exp_ma(df[ticker]['Close'], period=50) - 1
    
outcome_names = ['open_1', 'close_1', 'close_15']
col_names_out = pd.MultiIndex.from_product([tickers, outcome_names])
outcomes = pd.DataFrame(index=df.index, columns=col_names_out).sort_index()

for ticker in tickers:
    # next day's opening change
    outcomes.loc[:, (ticker, 'open_1')] = df[ticker]['Open'].shift(-1) / df[ticker]['Close'].shift(0) - 1
    
    # Change in closing price next day
    outcomes.loc[:, (ticker, 'close_1')] = df[ticker]['Close'].pct_change(-1)
    
    # Change in closing price in the next 15 days
    outcomes.loc[:, (ticker, 'close_15')] = df[ticker]['Close'].pct_change(-15)


In [ ]:
from sklearn.linear_model import LinearRegression


X = features['AAPL']
Y = outcomes['AAPL']['close_1']

# Get indices with NaN values
feat_zero_ind = features['AAPL'][features['AAPL'].isnull().any(1)].index
out_zero_ind = outcomes['AAPL'][outcomes['AAPL'].isnull().any(1)].index
zero_ind = np.concatenate([feat_zero_ind, out_zero_ind])

# Drop indices with NaN values
X, Y = X.drop(zero_ind), Y.drop(zero_ind)

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, shuffle=False)

model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print('RSQ in sample: '+str(model.score(X=X_train,y=y_train)))
print('RSQ out of sample: '+str(model.score(X=X_test,y=y_test)))
print('Regression Coefficients: '+str(model.coef_))


In [ ]:
plt.plot(y_test, label='Actual')
plt.plot(X_test.index, pred, label='Predicted')
plt.plot(X_test.index, X_test['DAILY_RETURN'], label='Daily return')
plt.legend()
plt.show()

## Feature Engineering - Technical Indicators

In [ ]:
# Daily return in percentage
df['DAILY_RETURN'] = ti.daily_return(df['Close'])

# Rate of change in volume
df['ROC'] = ti.rate_of_change(df['Volume'])

# Price difference
df['DIFF'] = ti.price_diff(df['Close'])

# Bollinger Bands
df[['BB_MID', 'BB_UPPER', 'BB_LOWER']] = ti.bollinger_bands(df['Close'])

# Relative Strengh Index (RSI)
df['RSI'] = ti.RSI(df['Close'])

# Moving Average Convergence Divergence (MACD)
df[['MACD', 'MACD_signal']] = ti.MACD(df['Close'])

# Stochastic Oscillator
df['STOCH'] = ti.stochastic_oscillator(df['Close'], df['High'], df['Low'])

# Accumulation/Distribution
df['ADI'] = ti.accumulation_distribution(df['Close'], df['Low'], df['High'], df['Volume'])

# Average True Range
df['ATR'] = ti.average_true_range(df['High'], df['Low'], df['Close'])

# Vortex
df[['VI_up', 'VI_down']] = ti.vortex(df['High'], df['Low'], df['Close'])

# Ease of Movement
df['EMV'] = ti.ease_of_movement(df['High'], df['Low'], df['Volume'])

# Commodity Channel Index
df['CCI'] = ti.commodity_channel(df['High'], df['Low'], df['Close'])

# On-Balance Volume
df['OBV'] = ti.on_balance_volume(df['Close'], df['Volume'])

Visualise daily return, rate of change and z-scores

In [ ]:
plt.figure(figsize=(20,10))

sns.histplot(df['DAILY_RETURN'], kde=True)
plt.show()

sns.histplot(df['ROC'], kde=True)
plt.show()




## Stock movement
Label column:

1 if daily return is > threshold

-1 if daily return is < threshold


In [ ]:
threshold = 0

df.loc[:, 'BUY_SIGNAL'] = 0
df.loc[:, 'SELL_SIGNAL'] = 0

df.loc[df['DAILY_RETURN'].shift(-1) > threshold, 'BUY_SIGNAL'] = 1
df.loc[df['DAILY_RETURN'].shift(-1) < -threshold, 'SELL_SIGNAL'] = 1

# Drop NaN values
df = df.dropna()

In [ ]:
plt.figure(figsize=(20,15))

plt.plot(X_train.index, X_train['DAILY_RETURN'], linewidth=1, label='Train data')
plt.plot(X_test.index, X_test['DAILY_RETURN'], linewidth=1, label='Data to predict')
plt.axhline(y=.0025, linewidth=1, color='g', linestyle='--', label='Threshold for buy signal')
plt.axhline(y=-.0025, linewidth=1, color='b', linestyle='--', label='Threshold for sell signal')

plt.legend()
plt.show()


In [ ]:
## Plot bollinger bands

plt.style.use('fivethirtyeight')

# Bollinger bands
plt.figure(figsize=(20,15))
plt.plot(df['Adj Close'], label='Adj Close', linewidth=1)
plt.fill_between(df['Adj Close'].index, df['BB_UPPER'], df['BB_LOWER'], alpha=.2)
plt.plot(df['BB_UPPER'], label='Upper band', linestyle='dashed', linewidth=2)
plt.plot(df['BB_LOWER'], label='Lower band', linestyle='dashed', linewidth=2)
plt.plot(df['BB_MID'], label='20-day simple moving average', linewidth=2)
plt.title('Bollinger bands')
plt.legend(fontsize=15)

plt.show()


In [ ]:
# Plotting MACD

plt.style.use('ggplot')

fig = plt.figure(facecolor = 'white', figsize = (20,10))

ax0 = plt.subplot2grid((12,8), (1,0), rowspan=6, colspan=4)
ax0.plot(df[['Adj Close','BB_MID']], linewidth=2)
ax0.set_facecolor('ghostwhite')
ax0.legend(['Adj Close','SMA'],ncol=3, loc = 'upper left', fontsize = 12)
plt.title("SPY Adj Close and MACD", fontsize = 20)

ax1 = plt.subplot2grid((12,8), (7,0), rowspan=3, colspan=4, sharex = ax0)
ax1.plot(df[['MACD', 'MACD_signal']], linewidth=1)
ax1.legend(['MACD', 'MACD_signal'], ncol=3, loc = 'lower left', fontsize = 12)
ax1.set_facecolor('silver')
plt.subplots_adjust(left=.09, bottom=.09, right=1, top=.95, wspace=.20, hspace=0)
plt.show()

In [ ]:
# Plotting OBV

fig = plt.figure(facecolor = 'white', figsize = (25,15))

ax0 = plt.subplot2grid((12,8), (1,0), rowspan=6, colspan=4)
ax0.plot(df[['Adj Close','BB_MID']], linewidth=2)
ax0.set_facecolor('ghostwhite')
ax0.legend(['Adj Close','SMA'],ncol=3, loc = 'upper left', fontsize = 12)
plt.title("SPY Adj Close and OBV", fontsize = 15)

ax1 = plt.subplot2grid((12,8), (7,0), rowspan=3, colspan=4, sharex = ax0)
ax1.plot(df['OBV'], linewidth=1)
ax1.legend(['OBV'], ncol=3, loc = 'upper left', fontsize = 12)
ax1.set_facecolor('silver')
plt.subplots_adjust(left=.09, bottom=.09, right=1, top=.95, wspace=.20, hspace=0)
plt.show()

In [ ]:
# Add moving averages for multiple periods
for period in [5, 10, 20, 50, 100, 200]:
    df['SMA{}'.format(str(period))] = ti.simple_ma(df['Adj Close'], length=period)
    df['EMA{}'.format(str(period))] = ti.exp_ma(df['Adj Close'], length=period)
    